In [1]:
from model_factory import create_model
import torch
import pandas as pd
from utils import preprocess_context
import pickle
from transformers import AutoTokenizer
from datasets import *
from torch.utils.data import DataLoader

device = torch.device('cuda:1' if not torch.cuda.is_available() else 'cpu')

In [2]:
model_type = 'roberta-ru'

model = create_model(model_type, 'bce', 13)

model_path = 'checkpoints/roberta_bs8_adam_gradclip_rocauctrack_bceloss_addfeat/epoch=2-step=942.ckpt'

def read_model(model, path):
    state_dict = torch.load(path)['state_dict']
    re_state_dict = {key.replace('model.', '', 1): value for key, value in state_dict.items()}
    model.load_state_dict(re_state_dict)
    return model

new_model = read_model(model, model_path)

csv_data = pd.read_csv('add_features_train.tsv', sep='\t')
csv_data.drop_duplicates(['context', 'phrase'], inplace=True)
csv_data['context'] = csv_data['context'].apply(lambda x: eval(x))
feature_cols = csv_data.columns[3:]

csv_data['prep_context'] = csv_data['context'].apply(lambda x: preprocess_context(x, 'roberta'))
csv_data['prep_phrase_context'] = csv_data['prep_context'] + '— ' + csv_data['phrase'] + '</s>'
csv_data = csv_data[['prep_phrase_context', 'label'] + list(feature_cols)]

with open('scaler.pickle', 'rb') as handle:
    scaler = pickle.load(handle)

text_data, labels, features = csv_data.values[:, 0], csv_data.values[:, 1], csv_data.values[:, 2:]
scaled_features = scaler.transform(features)

NameError: name 'create_model' is not defined

In [3]:
model_type = 'roberta-ru'

if model_type == 'bert': 
    hugg_name = 'sberbank-ai/ruBert-large'
elif model_type == 't5':
    hugg_name = 'sberbank-ai/ruT5-large'
elif model_type == 'roberta':
    hugg_name = 'xlm-roberta-large'
elif model_type == 'roberta-ru':
    hugg_name = 'DeepPavlov/xlm-roberta-large-en-ru'

tokenizer = AutoTokenizer.from_pretrained(hugg_name)

config = {'MAX_LEN': 512, 'tokenizer': tokenizer, 'model': model_type}

dataset = Dataset(text_data, 
                labels, 
                config=config,
                features=scaled_features
                )

In [1]:
dataset

NameError: name 'dataset' is not defined

In [17]:
def pred_label(dataset, model):
    batch = next(iter(dataset))
    ids = batch['ids']
    attention_mask = batch['att_mask']
    features = batch['features']
    out_probs = model.forward(input_ids=ids, attention_mask=attention_mask, features=features)
    pred_label = torch.where(out_probs > 0.5, 1, 0).item()
    return pred_label

In [16]:
pred_label = torch.where(out_probs > 0.5, 1, 0)
pred_label.item()

1

In [55]:
29000 * 0.15

4350.0

In [57]:
29000 * 0.85 / 3 * 1/2

4108.333333333333